In [5]:
import os
import pickle
import torch
import torchaudio
import json
# from asccd import encoding, preanalysis
import soundfile as sf
import numpy as np

from transformers import Data2VecAudioModel,Data2VecAudioConfig,Wav2Vec2Processor,AutoModel,AutoConfig
# from models.HiFiGAN_Generator import HiFiGAN

# Train SpeechAutoEncoder

## Prepare Dataset

In [14]:
from datasets import load_dataset, Audio
import datasets
# dataset_train = load_dataset("timit_asr", split="train")
# dataset_test = load_dataset("timit_asr", split="test")
# dataset = dataset_train.sort("id")
# dataset = dataset_test.sort("id")
# sampling_rate = dataset_train.features["audio"].sampling_rate

# processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
# small_train_dataset = dataset_train.map(prepare_dataset,remove_columns=dataset_test.column_names).select(range(1000))
# small_eval_dataset = dataset_test.map(prepare_dataset,remove_columns=dataset_test.column_names).select(range(100))


from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything
from torch.utils.data import DataLoader
import dataset 
import importlib
importlib.reload(dataset)

dataset = dataset.LibriSpeech(16, 16)
dataset.prepare_data()
dataset.setup()
# train_loader = dataset.test_dataloader()
# batch = next(iter(train_loader))
# dataset.dataset['train'][0]['audio']['array']


  0%|          | 0/4 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/338M [00:00<?, ?B/s]

ConnectionError: HTTPSConnectionPool(host='openslr.magicdatatech.com', port=443): Read timed out.

## Prepare Metrics and Models

In [ ]:
import importlib
import models.SpeechAutoEncoder as SAE
importlib.reload(SAE)

decoder_configuration = AutoConfig.from_pretrained("jaketae/hifigan-lj-v1", trust_remote_code=True,        
        upsample_rates=[2, 2, 2, 2, 2, 2, 5],
        upsample_initial_channel=768,
        upsample_kernel_sizes=[2, 2, 3, 3, 3, 3, 10],
        model_in_dim=768,
        sampling_rate=16000)
model = SAE.SpeechAutoEncoder(None,decoder_configuration,torch.nn.SmoothL1Loss())

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] ="1"
print(torch.cuda.is_available())
seed_everything(42)
torch.cuda.empty_cache()

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath="./experiments",
    filename='{epoch:02d}-{val_loss:.6f}',
    save_top_k=10,
    save_last=True,
    mode='min')
callbacks = [checkpoint_callback]
tb_logger = pl_loggers.TensorBoardLogger("./experiments")


trainer = Trainer(
    max_epochs=1000,
    gpus=1,
    callbacks=callbacks,
)
trainer.fit(model, dataset)

False


NameError: name 'seed_everything' is not defined